In [1]:
#import libraries
import os
import numpy as np
import pandas as pd
import cv2
import gc
import sys
from tqdm import tqdm
from glob import glob  


In [2]:
dirs = os.listdir('Dataset')
images_path = []                 #[]empty list
labels = []                      #[]empty list
for folder in dirs:
    #folder = 'Mask'             #checkstatment
    path = glob('./Dataset/{}/*.jpg'.format(folder))
    label = ['{}'.format(folder)]*len(path)
    #print(dirs)
    
    #appending
    images_path.extend(path)
    labels.extend(label)

In [3]:
#print(label)                      #Checkstatment
img_path = images_path[1]
img = cv2.imread(img_path)

In [4]:
cv2.imshow('original',img)
cv2.waitKey()
cv2.destroyAllWindows()

In [5]:
#loading face detection models
face_detection_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt', './models/res10_300x300_ssd_iter_140000_fp16.caffemodel')

In [6]:
def face_detection_dnn(img):
    # calculating blob from image (RGB mean subtraction from image)
    image = img.copy()
    h, w = image.shape[:2]

    blob = cv2.dnn.blobFromImage(image,1,(300, 300),(104, 117, 123), swapRB=True)     # (image[,scalefactor[,sizw[,mean[,swapRB,crop[,depth]]]]]

    # getting detections  passing blob as input to face_detection_model         
    face_detection_model.setInput(blob)
    detections = face_detection_model.forward()
    #print(detections)
    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2]    #confidence score
        #print(confidence)                  #checkstatment
        if confidence > 0.5:
            #print(confidence)
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])       #detections[0,0,i,3:7] give you bounding box info but in normalized form to de normalize * np.array([w,h,w,h]) multiply with W & H
            box = box.astype(int)
            #print(box)         #checkstatment      
            pt1 = (box[0],box[1])        #diagonal point of bounding box
            pt2 = (box[2],box[3])        #diagonal point of bounding box
            #cv2.rectangle(image,pt1,pt2,(0,255,0),2)       #colour,thickness_values     #checkstatment
            roi = image[box[1]:box[3],box[0]:box[2]]

            return roi 
    return None        

    #cv2.imshow('detected face',image)               #checkstatment
    #cv2.imshow('roi',roi)                           #checkstatment
    #cv2.waitKey(0)                                  #checkstatment
    #cv2.destroyAllWindows()                         #checkstatment      

In [7]:
#detections.shape            #checkstatment
img_roi = face_detection_dnn(img)

In [8]:
cv2.imshow('roi',img_roi)
cv2.imshow('original',img)
cv2.waitKey()
cv2.destroyAllWindows()

In [9]:
def datapreprocess(img):
    #blob from image(RGB mean subtraction from image)
    face = face_detection_dnn(img)
    if face is not None:      #all values will not be roi some will be none 
        #computing blob from image 
        blob = cv2.dnn.blobFromImage(face,1,(100,100),(104,117,123),swapRB=True)  #(image[,scalefactor[,sizw[,mean[,swapRB,crop[,depth]]]]]])   #if face any mem error reduce 100,100 to 80,80
        #print(blob.shape) #o/p: returing was on 4D so we need to squeeze to 3D   #checkstatment
        blob_squeeze = np.squeeze(blob).T  
        #print(blob_squeeze.shape)  #o/p is in 3D #checkstatment   
        blob_rotate = cv2.rotate(blob_squeeze,cv2.ROTATE_90_CLOCKWISE)
        
        blob_flip = cv2.flip(blob_rotate,1)

        #removing the negative values and normalizing the values
        img_norm = np.maximum(blob_flip,0)/blob_flip.max()      #0is the cutoff value.values below 0 are set to 0

        return img_norm
    else:
        return None    


    #cv2.imshow('roi',face)                                                     #checkstatment
    #cv2.namedWindow('blob',cv2.WINDOW_NORMAL)  #for_resizeing_blob_window      #checkstatment
    #cv2.namedWindow('blob_rotate',cv2.WINDOW_NORMAL)
    #cv2.namedWindow('blob_flip',cv2.WINDOW_NORMAL)                             #checkstatment
    #cv2.imshow('blob',blob_squeeze)                                            #checkstatment
    #cv2.imshow('blob_rotate',blob_rotate)
    #cv2.imshow('blob_flip',blob_flip)
    #cv2.waitKey()
    #cv2.destroyAllWindows()

In [10]:
#applying to all images
#len(images_path)       #checkstatment

data_img = []
labels_img = []
i = 0
for path,label in tqdm(zip(images_path,labels),desc='preprocessing'):  #to_know_eq_seq_No:_we_use tqdm
    img = cv2.imread(path)
    process_img = datapreprocess(img)
    if process_img is not None:
        data_img.append(process_img)
        labels_img.append(label)
    i += 1    #i=i+1    
    if i%100 == 0:
        gc.collect()                                                  #approx_time 170min 30.2sec


preprocessing: 134782it [2:41:55, 13.87it/s]


In [11]:
#blob_flip.max() , blob_flip.min()                      #there must not be negative value
x = np.array(data_img)
y = np.array(labels_img)

In [12]:
x.shape,y.shape

((134097, 100, 100, 3), (134097,))

In [13]:
np.savez('./Dataset/data_preprocess.npz',x,y)               #approx time 21min23sec